# Stock Investment Strategy Assessment

## In this excercise I analyzed a dataset containing daily stock data for over 5000 stocks over 5 years and find all ‘golden crosses’(when 50MA crosses above the 200MA). After finding all golden crosses (first instance for each stock was used for analysis) the next task was to find what percentage of those golden crosses result in a price increase by more than 1%, 2%, 3%, 4%, 5%, 10%, 15%, 20% over a week, two weeks, three weeks, a month, two months, three months, four months, five months, six months time intervals. The same exercise was done in the inverse to calulate losses following 'death crosses' (when 50DMA crosses below the 200DMA). This solution is unoptimized with opportunity to create greater efficiency populating the matrix used for  analysis. 

In [1]:
!pip install gcsfs

     |████████████████████████████████| 145 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 22.3 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 45.3 MB/s eta 0:00:01
     |████████████████████████████████| 338 kB 41.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 39.0 MB/s  eta 0:00:01
     |████████████████████████████████| 151 kB 28.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 34.2 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 223 kB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 76.9 MB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.3.0
    Uninstalling fsspec-2022.3.0:
      Successfully uninstalled fsspec-2022.3.0


In [2]:
!pip install dask

In [1]:
import pandas as pd
import numpy as np
import gcsfs as gc
import seaborn as sn
import matplotlib.pyplot as plt
import dask.dataframe as dd

In [2]:
# Pulling in data from google cloud storage bucket
df = dd.read_csv('https://storage.googleapis.com/case2-bucko/fh_5yrs.csv')

In [3]:
stocks = df.compute()

In [4]:
# Adding 50 Moving Average and 200 Moving Average columns to dataframe
stocks['50-EMA'] = stocks.groupby('symbol')['adjclose'].transform(lambda x: x.rolling(50, 50).mean())
stocks['200-EMA'] = stocks.groupby('symbol')['adjclose'].transform(lambda x: x.rolling(200, 200).mean())

In [5]:
# Dropping NAs so Moving Averages have same starting point
stocks.dropna(inplace = True)

In [6]:
# Adding golden cross indicator column
stocks["g_cross"] = stocks.apply(lambda row: 1 if row["50-EMA"] > row["200-EMA"]  else 0, axis=1)

In [7]:
# Adding death cross indicator column
stocks["d_cross"] = stocks.apply(lambda row: 1 if row["50-EMA"] < row["200-EMA"]  else 0, axis=1)

In [8]:
# Identiftying the index locations where first golden cross occurs and placing the index value into a list
stocks2 = stocks.copy()
stocks2['g_cross'].replace(0, np.nan,inplace=True)
first_cross_g = stocks2.dropna(subset = ['g_cross']).drop_duplicates(subset = ['symbol','g_cross'], keep='last')
g_cross_index = list(first_cross_g.index.values)

In [51]:
# Getting the index locations where first crossing occurs and placing the index value into a list
first_cross_g = stocks2.dropna(subset = ['g_cross']).drop_duplicates(subset = ['symbol','g_cross'], keep='last')
g_cross_index = list(first_cross_g.index.values)

In [9]:
# Identiftying the index locations where first death cross occurs and placing the index value into a list
stocks3 = stocks.copy()
stocks3['d_cross'].replace(0, np.nan,inplace=True)
first_cross_d = stocks3.dropna(subset = ['d_cross']).drop_duplicates(subset = ['symbol','d_cross'], keep='last')
d_cross_index = list(first_cross_d.index.values)

In [10]:
# Creating empty matrix to populate our increase overtime occurances
dummy_data = np.zeros((9,8), dtype = int)
percentage_gains = pd.DataFrame(dummy_data, columns = ['1%','2%','3%','4%','5%','10%','15%','20%'])


In [11]:
# For Loop: 1 week
# For loop comparing close price values at different incriments to place into our matrix
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][0] = percentage_gains['1%'][0] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][0] = percentage_gains['2%'][0] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][0] = percentage_gains['3%'][0] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][0] = percentage_gains['4%'][0] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][0] = percentage_gains['5%'][0] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][0] = percentage_gains['10%'][0] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][0] = percentage_gains['15%'][0] + 1
    elif (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][0] = percentage_gains['20%'][0] + 1
    

In [12]:
# For Loop: 2 Weeks
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][1] = percentage_gains['1%'][1] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][1] = percentage_gains['2%'][1] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][1] = percentage_gains['3%'][1] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][1] = percentage_gains['4%'][1] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][1] = percentage_gains['5%'][1] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][1] = percentage_gains['10%'][1] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][1] = percentage_gains['15%'][1] + 1
    elif (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][1] = percentage_gains['20%'][1] + 1

In [13]:
# For Loop: 3 Weeks
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][2] = percentage_gains['1%'][2] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][2] = percentage_gains['2%'][2] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][2] = percentage_gains['3%'][2] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][2] = percentage_gains['4%'][2] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][2] = percentage_gains['5%'][2] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][2] = percentage_gains['10%'][2] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][2] = percentage_gains['15%'][2] + 1
    elif (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][2] = percentage_gains['20%'][2] + 1

In [14]:
# For Loop: 1 month
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][3] = percentage_gains['1%'][3] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][3] = percentage_gains['2%'][3] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][3] = percentage_gains['3%'][3] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][3] = percentage_gains['4%'][3] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][3] = percentage_gains['5%'][3] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][3] = percentage_gains['10%'][3] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][3] = percentage_gains['15%'][3] + 1
    elif (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][3] = percentage_gains['20%'][3] + 1

In [15]:
# For Loop: 2 months
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][4] = percentage_gains['1%'][4] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][4] = percentage_gains['2%'][4] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][4] = percentage_gains['3%'][4] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][4] = percentage_gains['4%'][4] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][4] = percentage_gains['5%'][4] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][4] = percentage_gains['10%'][4] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][4] = percentage_gains['15%'][4] + 1
    elif (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][4] = percentage_gains['20%'][4] + 1

In [16]:
# For Loop: 3 Months
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][5] = percentage_gains['1%'][5] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][5] = percentage_gains['2%'][5] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][5] = percentage_gains['3%'][5] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][5] = percentage_gains['4%'][5] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][5] = percentage_gains['5%'][5] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][5] = percentage_gains['10%'][5] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][5] = percentage_gains['15%'][5] + 1
    elif (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][5] = percentage_gains['20%'][5] + 1

In [17]:
# For Loop: 4 Months
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][6] = percentage_gains['1%'][6] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][6] = percentage_gains['2%'][6] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][6] = percentage_gains['3%'][6] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][6] = percentage_gains['4%'][6] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][6] = percentage_gains['5%'][6] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][6] = percentage_gains['10%'][6] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][6] = percentage_gains['15%'][6] + 1
    elif (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][6] = percentage_gains['20%'][6] + 1

In [18]:
# For Loop: 5 Months
for i in g_cross_index:
    if .01 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][7] = percentage_gains['1%'][7] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][7] = percentage_gains['2%'][7] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][7] = percentage_gains['3%'][7] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][7] = percentage_gains['4%'][7] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][7] = percentage_gains['5%'][7] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][7] = percentage_gains['10%'][7] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][7] = percentage_gains['15%'][7] + 1
    elif (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][7] = percentage_gains['20%'][7] + 1

In [19]:
# For Loop: 6 Months
for i in g_cross_index:
    if .01 <= (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .02 :
        percentage_gains['1%'][8] = percentage_gains['1%'][8] + 1
    elif .02 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .03:
        percentage_gains['2%'][8] = percentage_gains['2%'][8] + 1
    elif .03 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .04:
        percentage_gains['3%'][8] = percentage_gains['3%'][8] + 1
    elif .04 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .05:
        percentage_gains['4%'][8] = percentage_gains['4%'][8] + 1
    elif .05 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .1:
        percentage_gains['5%'][8] = percentage_gains['5%'][8] + 1
    elif .1 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .15:
        percentage_gains['10%'][8] = percentage_gains['10%'][8] + 1
    elif .15 < (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] <= .20:
        percentage_gains['15%'][8] = percentage_gains['15%'][8] + 1
    elif (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] > .20:
        percentage_gains['20%'][8] = percentage_gains['20%'][8] + 1

In [20]:
# Creating empty matrix to populate our decrease overtime occurances
percentage_loss = pd.DataFrame(dummy_data, columns = ['1%','2%','3%','4%','5%','10%','15%','20%'])

In [21]:
# For Loop: 1 week
# For loop comparing close price values at different incriments to place into our matrix
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][0] = percentage_loss['1%'][0] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][0] = percentage_loss['2%'][0] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][0] = percentage_loss['3%'][0] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][0] = percentage_loss['4%'][0] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][0] = percentage_loss['5%'][0] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][0] = percentage_loss['10%'][0] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][0] = percentage_loss['15%'][0] + 1
    elif (stocks['adjclose'].iloc[i - 7] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][0] = percentage_loss['20%'][0] + 1

In [22]:
# For Loop: 2 weeks
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][1] = percentage_loss['1%'][1] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][1] = percentage_loss['2%'][1] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][1] = percentage_loss['3%'][1] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][1] = percentage_loss['4%'][1] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][1] = percentage_loss['5%'][1] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][1] = percentage_loss['10%'][1] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][1] = percentage_loss['15%'][1] + 1
    elif (stocks['adjclose'].iloc[i - 14] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][1] = percentage_loss['20%'][1] + 1

In [23]:
# For Loop: 3 weeks
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][2] = percentage_loss['1%'][2] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][2] = percentage_loss['2%'][2] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][2] = percentage_loss['3%'][2] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][2] = percentage_loss['4%'][2] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][2] = percentage_loss['5%'][2] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][2] = percentage_loss['10%'][2] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][2] = percentage_loss['15%'][2] + 1
    elif (stocks['adjclose'].iloc[i - 21] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][2] = percentage_loss['20%'][2] + 1

In [24]:
# For Loop: 1 month
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][3] = percentage_loss['1%'][3] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][3] = percentage_loss['2%'][3] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][3] = percentage_loss['3%'][3] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][3] = percentage_loss['4%'][3] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][3] = percentage_loss['5%'][3] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][3] = percentage_loss['10%'][3] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][3] = percentage_loss['15%'][3] + 1
    elif (stocks['adjclose'].iloc[i - 30] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][3] = percentage_loss['20%'][3] + 1

In [25]:
# For Loop: 2 months
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][4] = percentage_loss['1%'][4] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][4] = percentage_loss['2%'][4] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][4] = percentage_loss['3%'][4] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][4] = percentage_loss['4%'][4] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][4] = percentage_loss['5%'][4] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][4] = percentage_loss['10%'][4] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][4] = percentage_loss['15%'][4] + 1
    elif (stocks['adjclose'].iloc[i - 60] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][4] = percentage_loss['20%'][4] + 1

In [26]:
# For Loop: 3 months
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][5] = percentage_loss['1%'][5] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][5] = percentage_loss['2%'][5] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][5] = percentage_loss['3%'][5] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][5] = percentage_loss['4%'][5] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][5] = percentage_loss['5%'][5] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][5] = percentage_loss['10%'][5] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][5] = percentage_loss['15%'][5] + 1
    elif (stocks['adjclose'].iloc[i - 90] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][5] = percentage_loss['20%'][5] + 1

In [27]:
# For Loop: 4 months
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][6] = percentage_loss['1%'][6] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][6] = percentage_loss['2%'][6] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][6] = percentage_loss['3%'][6] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][6] = percentage_loss['4%'][6] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][6] = percentage_loss['5%'][6] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][6] = percentage_loss['10%'][6] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][6] = percentage_loss['15%'][6] + 1
    elif (stocks['adjclose'].iloc[i - 120] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][6] = percentage_loss['20%'][6] + 1

In [28]:
# For Loop: 5 months
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][7] = percentage_loss['1%'][7] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][7] = percentage_loss['2%'][7] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][7] = percentage_loss['3%'][7] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][7] = percentage_loss['4%'][7] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][7] = percentage_loss['5%'][7] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][7] = percentage_loss['10%'][7] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][7] = percentage_loss['15%'][7] + 1
    elif (stocks['adjclose'].iloc[i - 150] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][7] = percentage_loss['20%'][7] + 1

In [29]:
# For Loop: 6 months
for i in d_cross_index:
    if -.01 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.02:
        percentage_loss['1%'][8] = percentage_loss['1%'][8] + 1
    elif -.02 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.03:
        percentage_loss['2%'][8] = percentage_loss['2%'][8] + 1
    elif -.03 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.04:
        percentage_loss['3%'][8] = percentage_loss['3%'][8] + 1
    elif -.04 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.05:
        percentage_loss['4%'][8] = percentage_loss['4%'][8] + 1
    elif -.05 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.1:
        percentage_loss['5%'][8] = percentage_loss['5%'][8] + 1
    elif -.1 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.15:
        percentage_loss['10%'][8] = percentage_loss['10%'][8] + 1
    elif -.15 > (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] >= -.20:
        percentage_loss['15%'][8] = percentage_loss['15%'][8] + 1
    elif (stocks['adjclose'].iloc[i - 180] - stocks['adjclose'].iloc[i])/stocks['adjclose'].iloc[i] < -.20:
        percentage_loss['20%'][8] = percentage_loss['20%'][8] + 1

In [77]:
idx = ['1 week', '2 weeks', '3 weeks', '1 month', '2 months', '3 months', '4 months', '5 months', '6 months']

percentage_gains.index = idx 
percentage_gains2.index = idx
percentage_loss.index = idx
percentage_loss2.index = idx

In [78]:
percentage_gains

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,517,380,274,242,520,171,68,107
2 weeks,401,338,279,230,618,271,119,245
3 weeks,367,274,260,233,678,329,143,335
1 month,306,248,262,216,688,330,201,450
2 months,216,200,171,174,624,461,237,763
3 months,163,146,154,143,570,447,278,997
4 months,152,120,121,114,547,392,273,1207
5 months,120,116,117,113,416,360,324,1389
6 months,99,91,109,83,385,359,292,1541


In [79]:
percentage_loss

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,431,346,245,176,1075,197,54,73
2 weeks,301,287,230,204,1271,284,129,163
3 weeks,267,262,206,204,1310,316,187,261
1 month,264,197,200,156,1295,350,195,394
2 months,142,150,129,159,1124,380,228,710
3 months,134,127,114,105,1077,338,247,913
4 months,116,122,102,99,989,307,237,1097
5 months,125,117,95,90,771,299,227,1208
6 months,97,97,71,86,744,249,229,1296


In [57]:
len(first_cross_g['symbol'])

5667

In [56]:
len(first_cross_d['symbol'])

5693

In [47]:
import seaborn as sn
import matplotlib.pyplot as plt

In [80]:
percentage_gains.style.background_gradient(cmap='Greens', vmin=68, vmax=1541)

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,517,380,274,242,520,171,68,107
2 weeks,401,338,279,230,618,271,119,245
3 weeks,367,274,260,233,678,329,143,335
1 month,306,248,262,216,688,330,201,450
2 months,216,200,171,174,624,461,237,763
3 months,163,146,154,143,570,447,278,997
4 months,152,120,121,114,547,392,273,1207
5 months,120,116,117,113,416,360,324,1389
6 months,99,91,109,83,385,359,292,1541


In [81]:
percentage_loss.style.background_gradient(cmap='Blues', vmin=54, vmax=1310)

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,431,346,245,176,1075,197,54,73
2 weeks,301,287,230,204,1271,284,129,163
3 weeks,267,262,206,204,1310,316,187,261
1 month,264,197,200,156,1295,350,195,394
2 months,142,150,129,159,1124,380,228,710
3 months,134,127,114,105,1077,338,247,913
4 months,116,122,102,99,989,307,237,1097
5 months,125,117,95,90,771,299,227,1208
6 months,97,97,71,86,744,249,229,1296


In [60]:
percentage_gains2 = percentage_gains / 5667
percentage_loss2 = percentage_loss / 5693

In [82]:
# Heatmapped DataFrame displaying concentration of stock gains
percentage_gains2.style.background_gradient(cmap='Greens', vmin = 0.011999, vmax =0.271925)

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,0.091230,0.067055,0.048350,0.042703,0.091759,0.030175,0.011999,0.018881
2 weeks,0.070761,0.059644,0.049232,0.040586,0.109052,0.047821,0.020999,0.043233
3 weeks,0.064761,0.048350,0.045880,0.041115,0.119640,0.058055,0.025234,0.059114
1 month,0.053997,0.043762,0.046233,0.038115,0.121405,0.058232,0.035469,0.079407
2 months,0.038115,0.035292,0.030175,0.030704,0.110111,0.081348,0.041821,0.134639
3 months,0.028763,0.025763,0.027175,0.025234,0.100582,0.078878,0.049056,0.175931
4 months,0.026822,0.021175,0.021352,0.020116,0.096524,0.069172,0.048174,0.212987
5 months,0.021175,0.020469,0.020646,0.019940,0.073407,0.063526,0.057173,0.245103
6 months,0.017470,0.016058,0.019234,0.014646,0.067937,0.063349,0.051526,0.271925


In [83]:
# Heatmapped DataFrame displaying concentration of stock losses
percentage_loss2.style.background_gradient(cmap='Blues', vmin =0.009485 , vmax = 0.230107)

,1%,2%,3%,4%,5%,10%,15%,20%
1 week,0.075707,0.060776,0.043035,0.030915,0.188828,0.034604,0.009485,0.012823
2 weeks,0.052872,0.050413,0.040400,0.035833,0.223257,0.049886,0.022659,0.028632
3 weeks,0.046900,0.046021,0.036185,0.035833,0.230107,0.055507,0.032847,0.045846
1 month,0.046373,0.034604,0.035131,0.027402,0.227472,0.061479,0.034253,0.069208
2 months,0.024943,0.026348,0.022659,0.027929,0.197435,0.066749,0.040049,0.124715
3 months,0.023538,0.022308,0.020025,0.018444,0.189180,0.059371,0.043387,0.160372
4 months,0.020376,0.021430,0.017917,0.017390,0.173722,0.053926,0.041630,0.192693
5 months,0.021957,0.020552,0.016687,0.015809,0.135429,0.052521,0.039874,0.212190
6 months,0.017038,0.017038,0.012471,0.015106,0.130687,0.043738,0.040225,0.227648
